![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/derive-column-by-example.png)

# Derive Column By Example


One of the more advanced tools in Data Prep is the ability to derive columns by providing examples of desired results and letting Data Prep generate code to achieve the intended derivation.

In [ ]:
import azureml.dataprep as dprep

In [ ]:
dflow = dprep.read_csv(path = '../data/crime-spring.csv')
df = dflow.head(5)
df

As you can see, this is a fairly simple file, but let's assume that we need to be able to join this with a dataset where date and time come in a format 'Apr 4, 2016 | 10PM-12AM'.

Let's wrangle the data into the shape we need.

In [ ]:
builder = dflow.builders.derive_column_by_example(source_columns = ['Date'], new_column_name = 'date_timerange')
builder.add_example(source_data = df.iloc[0], example_value = 'Apr 4, 2016 10PM-12AM')
builder.preview() # will preview top 10 rows

The code above first creates a builder for the derived column by providing an array of source columns to consider ('DATE') and name for the new column to be added.

Then, we provide the first example by passing in the first row (index 0) of the DataFrame printed above and giving an expected value for the derived column.

Finally, we call `builder.preview()` and observe the derived column next to the source column.

Everything looks good here. However, we just noticed that it's not quite what we wanted. We forgot to separate date and time range by '|' to generate the format we need.

To fix that, we will add another example. This time, instead of passing in a row from the preview, we just construct a dictionary of column name to value for the source_data parameter.

In [ ]:
builder.add_example(source_data = {'Date': '4/15/2016 10:00'}, example_value = 'Apr 15, 2016 | 10AM-12PM')
builder.preview()

This clearly had negative effects, as now the only rows that have any values in derived column are the ones that match exactly with the examples we have provided.

Let's look at the examples:

In [ ]:
examples = builder.list_examples()
examples

Here we can see that we have provided inconsistent examples. To fix the issue, we need to replace the first example with a correct one (including '|' between date and time).

We can achieve this by deleting examples that are incorrect (by either passing in example_row from examples DataFrame, or by just passing in example_id value) and then adding new modified examples back.

In [ ]:
builder.delete_example(example_id = -1)
builder.add_example(examples.iloc[0], 'Apr 4, 2016 | 10PM-12AM')
builder.preview()

Now this looks correct and we can finally call to_dataflow() on the builder, which would return a dataflow with the desired derived columns added.

In [ ]:
dflow = builder.to_dataflow()

In [ ]:
df = dflow.to_pandas_dataframe()
df